In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import os
import numpy as np
import pandas as pd
from datetime import datetime


#DATOS PISOA
#flat="./pisoA/"
#day="19244"
#folders=["TRACK2","TRACK3","TRACK4","TRACK5","TRACK6","TRACK7","TRACK8","TRACK9"]

#DATOS PISOB
flat="./data-iphone/"
folders=["track1","track2","track3","track4","track5"]


#Estas variables definen los intervalos de tiempo que segmentan los datos para crear una secuencia de valores RSSI a partir de un tiempo determinado
#Ventanas temporales de valores "futuros" (los datos RSSI despues de la estimación de tiempo) hasta 12 segundos
tW2=[6,4,3,2,1,0]
#Ventanas temporales de valores "pasados" (los datos RSSI despues de la estimación de tiempo) hasta 12 segundos
tW1=[0,-1,-2,-3,-4,-6]


file_folder={}
for f in folders:
    for path in os.listdir(flat+f):
        # check if current path is a file
        if os.path.isfile(os.path.join(flat+f, path)) and not path.endswith("_coords"):
            file_folder[f]=path
            
print(file_folder)
        

#LEEMOS los datos RSSI (tiempo, ancla y señal)
data=[]
N=0
for f in folders:
    series=read_csv(flat+f+"/"+file_folder[f], sep=" ",parse_dates=True,header=None)
    N=N+len(series.values)
    d2=series.values
    d2=np.array(d2)
    print(d2)
    data.extend(d2)

data=np.array(data)
print(len(data))
print(N)


{'track1': '1673279878.461661', 'track2': '1673280059.055518', 'track3': '1673280138.201364', 'track4': '1673280228.433104', 'track5': '1673280328.044337'}
[[1673279854.539098 1.3546343 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  'e368d11b55bdede71777a196ddb32207']
 [1673279854.539139 1.3546343 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  'e368d11b55bdede71777a196ddb32207']
 [1673279854.539159 1.3546343 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  'e368d11b55bdede71777a196ddb32207']
 ...
 [1673279943.942491 1.1160754 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  '8cf3ea91b8b802892f7045836c112e0a']
 [1673279944.259109 1.1297523 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  '8cf3ea91b8b802892f7045836c112e0a']
 [1673279944.259142 1.1297523 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  '8cf3ea91b8b802892f7045836c112e0a']]
[[1673280059.5118918 1.6682537 0.0 'B0C87CEA-B8A4-498F-9744-E434105F0F6A'
  'fd062cd54ac5cc7f1ab7efbb0b753023']
 [1673280059.511983 0.08381477 0.0 'B0C87CEA-B8A4-498F-9744-

In [2]:
#Leemos los datos del ground truth que ha etiquetado el usuario (pixeles en la imagen)
dataY=[]

for f in folders:
    series=read_csv(flat+f+"/"+file_folder[f]+"_coords", sep=" ",parse_dates=True,header=None)
    for d in series.values:
        dataY.append(d)
dataY=np.array(dataY)

In [3]:
#Leemos las diferentes anclas (4) que hay configuradas
print(data[:,4])
anchors=np.unique(data[:,4])
print(anchors)

['e368d11b55bdede71777a196ddb32207' 'e368d11b55bdede71777a196ddb32207'
 'e368d11b55bdede71777a196ddb32207' ... 'e368d11b55bdede71777a196ddb32207'
 'e368d11b55bdede71777a196ddb32207' 'e368d11b55bdede71777a196ddb32207']
['8cf3ea91b8b802892f7045836c112e0a' 'e368d11b55bdede71777a196ddb32207'
 'fd062cd54ac5cc7f1ab7efbb0b753023']


In [4]:
#Normalizamos los valores de RSSI
min_v=(float)(min(data[:,1]))
max_v=(float)(max(data[:,1]))
print(min_v,max_v)

0.08381477 4.0543637


In [5]:
#INFO: tiempo inicial y final de los datos
min_t=(float)(min(dataY[:,0]))
max_t=(float)(max(dataY[:,0]))
print(min_t,max_t)

1673279881.5890388 1673280419.295923


In [6]:
#INFO: valores min max de la Y
min_y=(float)(min(dataY[:,2]))
max_y=(float)(max(dataY[:,2]))
print(min_y,max_y)

51.33332824707031 552.6666564941406


In [7]:
#INFO: valores min max de la X
min_x=(float)(min(dataY[:,1]))
max_x=(float)(max(dataY[:,1]))
print(min_x,max_x)

24.333328247070312 336.6666564941406


In [8]:
#Esta función obtiene la media de un cojunto de señales RSSI (normalizada)
def getAVG(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (np.mean(vs)-min_v)/(max_v-min_v)

#Esta función obtiene el MAX de un cojunto de señales RSSI (normalizada)
def getMAX(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (max(vs)-min_v)/(max_v-min_v)

#Esta función obtiene el MIN de un cojunto de señales RSSI (normalizada)
def getMIN(vs):
    if (len(vs)==0):
        return 0
    #print(" avg:", vs)
    return (min(vs)-min_v)/(max_v-min_v)

In [9]:
#Valores de X, Y
X=[]
Y=[]



c=0

#Desde el tiempo inicial al final (en segundos) de uno en uno (con un desplazamiento en funcion del tamaño de ventana para recoger datos anteriores y futuros)
for t in np.arange(min_t-min(tW1),max_t-max(tW2),1.0):
    print(t)
    
    #Recogemos la posicioón de X,Y más cercana anterior y posterior a este tiempo t
    agXY0=dataY[np.sort(np.where( (dataY[:,0] <  t)))][0][-1]
    agXYNn=dataY[np.sort(np.where( (dataY[:,0] >=  t)))][0][0]
    #calculamos el incremento de tiempo respecto t, del futuro y del pasado
    incT0=t-agXY0[0]
    incTN=agXYNn[0]-t
    print("agXY0:",agXY0, "incT0:",incT0)
    print("agXYNn:",agXYNn, "incTN:",incTN)
    
    #IMPORTANTE: si el usuario no etiquetó esa muestra en un tiempo cercano (a 10 segundos), la descartamos!
    if(incT0>5 or incTN> 5):
        print("NOT DATA AVAILABLE!")
        continue
    
    #valores de las x-> NO posicion, los datos de entrada de RSSI
    xi=[]
    for a in anchors:
        xirssi1=[]
        xirssi2=[]
        xirssi3=[]
        xirssi4=[]
        xirssi5=[]
        xirssi6=[]
        #Ventanas futuras
        for it in range(len(tW1)-2,-1,-1):
            t0=t+tW1[it]
            tN=t+tW1[it+1]
             #Todos los datos en ese segmento de ventana
            ag1=data[np.where((data[:,4] == a) & (data[:,0] <= t0) & (data[:,0] >=  tN))]
            #print("t+:",t,"a:",str(a),"t0:",t0,"tN:",tN," ag1:", ag1)
            #Sacamos las 3 características de esa ventana (max,min,media)
            xirssi1.append(getAVG(ag1[:,1]))
            xirssi2.append(getMAX(ag1[:,1]))
            xirssi3.append(getMIN(ag1[:,1]))
        #Ventanas pasadas
        for it in range(0,len(tW2)-1):
            t0=t+tW2[it]
            tN=t+tW2[it+1]
            ag1=data[np.where((data[:,4] == a) & (data[:,0] <= t0) & (data[:,0] >=  tN))]
            #print("t-:",t,"a:",str(a),"t0:",t0,"tN:",tN," ag1:", ag1)
            #Sacamos las 3 características de esa ventana (max,min,media)
            xirssi4.append(getAVG(ag1[:,1]))
            xirssi5.append(getMAX(ag1[:,1]))
            xirssi6.append(getMIN(ag1[:,1]))
            
        #añadimos los valores de entradas futuras y pasadas a la sequencia     
        xi.append(xirssi1)
        xi.append(xirssi2)
        xi.append(xirssi3)
        xi.append(xirssi4)
        xi.append(xirssi5)
        xi.append(xirssi6)
    

    
    #calculamos una posición X,Y de la imagen ponderada respecto las posiciones más cercana anterior y posterior a este tiempo t
    w0= 1-(incT0)/(incT0+incTN)
    wn= 1-(incTN)/(incT0+incTN)
    
    posX=(agXYNn[1]*wn+agXY0[1]*w0)
    posY=(agXYNn[2]*wn+agXY0[2]*w0)
    print("posX:",posX, "posY:",posY)
    
    #normalizamos 
    posX=(posX-min_x)/(max_x-min_x)
    posY=(posY-min_y)/(max_y-min_y)
    print("posX:",posX, "posY:",posY)
    
    #añadimos la sequencia de caracteristicas X (RSSI) e Y(posX,posY de la imagen)
    print("xi.shape:",np.array(xi).shape)
    X.append(np.transpose(xi))
    Y.append([posX,posY])
    

    
    
    print("count:",c)



    c=c+1
    if(c>5000):
        break
        
        
#Importante: pasar los datos a NUMPY para trabajar con el modelo de DL
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(Y.shape)


1673279887.5890388
agXY0: [1.67327989e+09 2.54666656e+02 4.82333328e+02] incT0: 2.1168859004974365
agXYNn: [1.67327989e+09 1.79666656e+02 5.22000000e+02] incTN: 3.350133180618286
posX: 225.62588671052646 posY: 497.6926706798573
posX: 0.6444799201967466 posY: 0.8903444420770872
xi.shape: (18, 5)
count: 0
1673279888.5890388
agXY0: [1.67327989e+09 2.54666656e+02 4.82333328e+02] incT0: 3.1168859004974365
agXYNn: [1.67327989e+09 1.79666656e+02 5.22000000e+02] incTN: 2.350133180618286
posX: 211.90725890125714 posY: 504.94830142933597
posX: 0.6005568848733525 posY: 0.9048171099422941
xi.shape: (18, 5)
count: 1
1673279889.5890388
agXY0: [1.67327989e+09 2.54666656e+02 4.82333328e+02] incT0: 4.1168859004974365
agXYNn: [1.67327989e+09 1.79666656e+02 5.22000000e+02] incTN: 1.3501331806182861
posX: 198.18863109198782 posY: 512.2039321788145
posX: 0.5566338495499584 posY: 0.919289777807501
xi.shape: (18, 5)
count: 2
1673279890.5890388
agXY0: [1.67327989e+09 2.54666656e+02 4.82333328e+02] incT0: 5.11

posX: 185.1499477841392 posY: 424.80336689084214
posX: 0.5148877977244086 posY: 0.7449535420867051
xi.shape: (18, 5)
count: 35
1673279924.5890388
agXY0: [1.67327992e+09 2.13000000e+02 4.24000000e+02] incT0: 0.4108278751373291
agXYNn: [1.67327993e+09 2.37666656e+02 3.76000000e+02] incTN: 2.0527000427246094
posX: 217.11351135936187 posY: 415.9953259455219
posX: 0.61722578308964 posY: 0.7273843112994406
xi.shape: (18, 5)
count: 36
1673279925.5890388
agXY0: [1.67327992e+09 2.13000000e+02 4.24000000e+02] incT0: 1.410827875137329
agXYNn: [1.67327993e+09 2.37666656e+02 3.76000000e+02] incTN: 1.0527000427246094
posX: 227.126248099747 posY: 396.5110734424456
posX: 0.6492836387036415 posY: 0.6885194455399594
xi.shape: (18, 5)
count: 37
1673279926.5890388
agXY0: [1.67327992e+09 2.13000000e+02 4.24000000e+02] incT0: 2.410827875137329
agXYNn: [1.67327993e+09 2.37666656e+02 3.76000000e+02] incTN: 0.052700042724609375
posX: 237.13898484013208 posY: 377.0268209393692
posX: 0.6813414943176429 posY: 0.6

posX: 198.75243389909866 posY: 481.5995047313429
posX: 0.5584389812990264 posY: 0.858243711800916
xi.shape: (18, 5)
count: 73
1673280072.5890388
agXY0: [1.67328007e+09 1.79666656e+02 5.36666656e+02] incT0: 2.736733913421631
agXYNn: [1.67328007e+09 2.20333328e+02 4.19333328e+02] incTN: 0.9637801647186279
posX: 209.74189938800373 posY: 449.8921998084448
posX: 0.5936240367991293 posY: 0.7949977571907091
xi.shape: (18, 5)
count: 74
1673280073.5890388
agXY0: [1.67328007e+09 2.20333328e+02 4.19333328e+02] incT0: 0.03621983528137207
agXYNn: [1.67328008e+09 1.49000000e+02 4.23000000e+02] incTN: 1.8305160999298096
posX: 218.94926457183567 posY: 419.4044718107726
posX: 0.6231033281559214 posY: 0.7341844693443302
xi.shape: (18, 5)
count: 75
1673280074.5890388
agXY0: [1.67328007e+09 2.20333328e+02 4.19333328e+02] incT0: 1.036219835281372
agXYNn: [1.67328008e+09 1.49000000e+02 4.23000000e+02] incTN: 0.8305160999298096
posX: 180.73639959101533 posY: 421.368687361439
posX: 0.5007569068012583 posY: 0.

posX: 117.03599857369038 posY: 83.14480009462196
posX: 0.2968068468610174 posY: 0.06345373438223542
xi.shape: (18, 5)
count: 115
1673280114.5890388
agXY0: [1.67328011e+09 1.23666656e+02 8.36666565e+01] incT0: 1.250213861465454
agXYNn: [1.67328012e+09 5.16666565e+01 7.80000000e+01] incTN: 1.466771125793457
posX: 90.53603617506161 posY: 81.0591586498904
posX: 0.21196171506750586 posY: 0.05929354528803722
xi.shape: (18, 5)
count: 116
1673280115.5890388
agXY0: [1.67328011e+09 1.23666656e+02 8.36666565e+01] incT0: 2.250213861465454
agXYNn: [1.67328012e+09 5.16666565e+01 7.80000000e+01] incTN: 0.46677112579345703
posX: 64.03607377643286 posY: 78.97351720515884
posX: 0.12711658327399442 posY: 0.05513335619383899
xi.shape: (18, 5)
count: 117
1673280116.5890388
agXY0: [1.67328012e+09 5.16666565e+01 7.80000000e+01] incT0: 0.533228874206543
agXYNn: [1.67328012e+09 5.43333282e+01 1.62666656e+02] incTN: 1.2502341270446777
posX: 52.46395163045436 posY: 103.31406925376595
posX: 0.09006603150955253 po

posX: 204.9141925325637 posY: 393.74130077407756
posX: 0.5781671309270122 posY: 0.6829946329805138
xi.shape: (18, 5)
count: 203
1673280237.5890388
agXY0: [1.67328024e+09 1.98333328e+02 3.87666656e+02] incT0: 1.1910068988800049
agXYNn: [1.67328024e+09 2.93666656e+02 4.75666656e+02] incTN: 1.5760040283203125
posX: 239.36773491287124 posY: 425.5445723603734
posX: 0.6884773004298108 posY: 0.7464320104584826
xi.shape: (18, 5)
count: 204
1673280238.5890388
agXY0: [1.67328024e+09 1.98333328e+02 3.87666656e+02] incT0: 2.191006898880005
agXYNn: [1.67328024e+09 2.93666656e+02 4.75666656e+02] incTN: 0.5760040283203125
posX: 273.82127729317887 posY: 457.3478439466693
posX: 0.7987874699326096 posY: 0.8098693879364515
xi.shape: (18, 5)
count: 205
1673280239.5890388
agXY0: [1.67328024e+09 2.93666656e+02 4.75666656e+02] incT0: 0.4239959716796875
agXYNn: [1.67328024e+09 2.99333328e+02 5.52666656e+02] incTN: 2.3594322204589844
posX: 294.5298531819011 posY: 487.39596566458874
posX: 0.8650902753518915 pos

posX: 149.3030292399928 posY: 215.9732614854358
posX: 0.400116445127065 posY: 0.3284041254828097
xi.shape: (18, 5)
count: 311
1673280372.5890388
agXY0: [1.67328037e+09 1.51000000e+02 2.38333328e+02] incT0: 0.43103480339050293
agXYNn: [1.67328037e+09 4.20000000e+01 8.90000000e+01] incTN: 2.3525400161743164
posX: 134.1214188175082 posY: 215.20915665009898
posX: 0.3515093672090938 posY: 0.32687998018409486
xi.shape: (18, 5)
count: 312
1673280373.5890388
agXY0: [1.67328037e+09 1.51000000e+02 2.38333328e+02] incT0: 1.431034803390503
agXYNn: [1.67328037e+09 4.20000000e+01 8.90000000e+01] incTN: 1.3525400161743164
posX: 94.9631396026384 posY: 161.56111845207533
posX: 0.226136005888224 posY: 0.21986926460768205
xi.shape: (18, 5)
count: 313
1673280374.5890388
agXY0: [1.67328037e+09 1.51000000e+02 2.38333328e+02] incT0: 2.431034803390503
agXYNn: [1.67328037e+09 4.20000000e+01 8.90000000e+01] incTN: 0.3525400161743164
posX: 55.8048603877686 posY: 107.91308025405161
posX: 0.1007626445673541 posY: 

In [10]:
print(X)
print(X.shape)


[[[0.9024279  0.9121351  0.88537328 ... 0.         0.         0.        ]
  [0.88009738 0.88025477 0.87994    ... 0.         0.         0.        ]
  [0.88822364 0.8909004  0.88296387 ... 0.         0.         0.        ]
  [0.89059832 0.89531168 0.88438238 ... 0.         0.         0.        ]
  [0.89338723 0.89338723 0.89338723 ... 0.         0.         0.        ]]

 [[0.89663484 0.9121351  0.87994    ... 0.11149862 0.11149862 0.11149862]
  [0.88822364 0.8909004  0.88296387 ... 0.         0.         0.        ]
  [0.89059832 0.89531168 0.88438238 ... 0.         0.         0.        ]
  [0.89338723 0.89338723 0.89338723 ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.88497314 0.8909004  0.87994    ... 0.14715278 0.17108823 0.11149862]
  [0.89059832 0.89531168 0.88438238 ... 0.         0.         0.        ]
  [0.89338723 0.89338723 0.89338723 ... 0.         0.         0.        ]
  [0.         0.         0.       

In [11]:
print(Y)
print(Y.shape)


[[0.64447992 0.89034444]
 [0.60055688 0.90481711]
 [0.55663385 0.91928978]
 [0.45689534 0.92232036]
 [0.39467248 0.89691599]
 [0.33244963 0.87151163]
 [0.27708589 0.8480837 ]
 [0.27369409 0.83963127]
 [0.27030229 0.83117884]
 [0.267522   0.78174289]
 [0.26515044 0.70491301]
 [0.26298282 0.63653489]
 [0.2608664  0.57027871]
 [0.3892129  0.60816067]
 [0.5497544  0.67414879]
 [0.5867649  0.71367634]
 [0.63805829 0.74486255]
 [0.74130892 0.74570525]
 [0.84455956 0.74654794]
 [0.87214626 0.77341879]
 [0.8862332  0.8049335 ]
 [0.90032015 0.83644822]
 [0.84273595 0.85098002]
 [0.71769093 0.84952659]
 [0.60979215 0.84771264]
 [0.56001563 0.8446766 ]
 [0.51023912 0.84164057]
 [0.47178828 0.84695072]
 [0.49200896 0.89549751]
 [0.51222964 0.9440443 ]
 [0.41361059 0.90705192]
 [0.31473485 0.86987477]
 [0.27031247 0.82372708]
 [0.27086354 0.77017055]
 [0.3635437  0.74767339]
 [0.5148878  0.74495354]
 [0.61722578 0.72738431]
 [0.64928364 0.68851945]
 [0.68134149 0.64965458]
 [0.58267028 0.64715112]


 [0.64163055 0.75949281]]
(353, 2)


In [ ]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.model_selection import KFold


#Dividimos los datos en K fold para cross validation
kf = KFold(n_splits=10,shuffle=True)
kf.get_n_splits(X)


#Un fichero para escribir los resultados de prediccion y reales ("borramos lo anterior")
f = open(flat+"xy.tsv", "w") 
f.write("")
f.close()


#Para cada K del cross validator   
for train_index, test_index in kf.split(X):
    #Datos de train y test
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_val = X[train_index], X[test_index]
    y_train, y_val = Y[train_index], Y[test_index]


    #un nuevo modelo
    model = keras.Sequential()

    #3 capas de convolucion 
    #INPUT (8 valores de cada sequencia, 4 anclas * 2 (ventanas futuras+ventana pasadas) * 3 caracteristicas (max,min,avg))
    model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(X.shape[1],X.shape[2])))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    #model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))

    #2 capas de LSTM de secuencia
    model.add(Dropout(0.5))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))

    #lo pasamos a una capa densa y hacemos regrension de dos valores (posX,posY)
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dense(256))
    model.add(Dense(2, activation="sigmoid"))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()

    #iteramos 1000 veces
    history = model.fit(x_train , y_train,
                        batch_size=64,
                validation_data=(x_val,y_val),
                epochs=300)
    # Predecimos
    yhat = model.predict(x_val)

    #Desnormalizamos
    x1=y_val[:,0]*(max_x-min_x)+min_x
    y1=y_val[:,1]*(max_y-min_y)+min_y
    x2=yhat[:,0]*(max_x-min_x)+min_x
    y2=yhat[:,1]*(max_y-min_y)+min_y

    #calculamos error absoluto y ponemos indice, posiciones reales y predichas 
    absx=0.0
    absy=0.0

    f = open(flat+"xy.tsv", "a")
    for i in range(0,len(x2)):
        f.write(str(test_index[i])+"\t"+str(x1[i])+"\t"+str(y1[i])+"\t"+str(x2[i])+"\t"+str(y2[i])+"\n")
        absx=absx+abs(x1[i]-x2[i])
        absy=absy+abs(y1[i]-y2[i])
    f.close()

    absx=absx/(float)(len(x2))
    absy=absy/(float)(len(x2))

    print("absx:"+str(absx))
    print("absy:"+str(absy))